In [2]:
#8.1 Changing the String Representation of Instances

class Pair:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __repr__(self):
        return 'Pair({0.a!r}, {0.b!r})'.format(self)
    def __str__(self):
        return '({0.a!s}, {0.b!s})'.format(self)

In [5]:
test1 = Pair(5, 6)
test1

Pair(5, 6)

In [11]:
print(test1)

(5, 6


In [8]:
print('test1 is {0!r}'.format(test1))

test1 is Pair(5, 6)


In [12]:
print('test1 is {0}'.format(test1))

test1 is (5, 6


In [25]:
#8.2 Customing String Formatting

_formats = {
    'nmy': '{s.name}-{s.major}-{s.year}',
    'ymn': '{s.year}/{s.major}/{s.name}'
}

class Student:
    def __init__(self, name, major, year):
        self.name = name
        self.major = major
        self.year = year
        
    def __format__(self, code):
        if code == '': #setting default to name-major-year if format is not explicitly stated
            code = 'nmy'
        fmt = _formats[code]
        return fmt.format(s=self)

In [26]:
student_a = Student('Jason Kidd', 'Freshmen', 'Business')
format(student_a)

'Jason Kidd-Freshmen-Business'

In [27]:
format(student_a, 'ymn')

'Business/Freshmen/Jason Kidd'

In [32]:
'Student info: {:ymn}'.format(student_a)

'Student info: Business/Freshmen/Jason Kidd'

In [37]:
#8.3 Making Objects Support the Context-Management Protocol

from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None
    
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Connection already established')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [39]:
from functools import partial

connection = LazyConnection(('www.wikipedia.org', 80))
#Connection closed

with connection as s: #conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.wikipedia.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b'')) #conn.__exit__() executes: connection closed

In [40]:
#8.4 Saving Memory When Creating a Large Number of Instances

class Student:
    __slots__=['name', 'major', 'year']
    def __init__(self, name, major, year):
        self.name = name
        self.major = major
        self.year = year

In [ ]:
#8.5 Encapsulating Names in a Class

class X:
    def __init__(self):
        self._internal = 0  # an internal attribute
        self.public = 1     # A public attribute
        
    def public_method(self):
        ```
        A public method
        ```

    def _internal_method(self):
        ```
        An internal method
        ```

In [ ]:
class Y:
    def __init__(self):
        self.__private = 0
    def __private_method(self):
        ```
        A private method
        ```
    def public_method(self):
        ```
        self.__private_method()
        ```

In [ ]:
#private attributes in the preceding class cannot be overridden via inheritance

class Z(X):
    def __init__(self):
        super().__init__()
        self.__private = 1   #Does not override X.__private
        
    def __private_method(self):  #Does not override X.__private_method()
        ```
        ```

In [54]:
#8.6 Creating Managed Attributes

class Team:
    def __init__(self, team_name):
        self.team_name = team_name
    
    #Getter function
    @property
    def team_name(self):
        return self._team_name
    
    #Setter function
    @team_name.setter
    def team_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._team_name = value
        
    #Deleter function (optional) 
    @team_name.deleter
    def team_name(self):
        raise AttributeError('Cannot delete attribute')

In [55]:
team1 = Team('Los Angles Angels')
team1.team_name

'Los Angles Angels'

In [56]:
team2 = Team('Houston Astros')
team2.team_name

'Houston Astros'

In [58]:
del team2.team_name

AttributeError: Cannot delete attribute

In [49]:
#8.7 Calling a Method on a Parent Class

class A:
    def spam(self):
        print('A.spam')
        
class A(B):
    def spam(self):
        print('B.spam')
        super().spam()   #call parent spam()

In [50]:
class A:
    def __init__(self):
        self.x = 0
        
class B(A):
    def __init__(self):
        super().__init__() #making sure __init__() is properly initialized
        self.y = 1

In [51]:
class Proxy:
    def __init__(self,obj):
        self._obj = obj
        
    def __getattr__(self, name):
        return getattr(self._obj, name) # Delegate attribute lookup to internal obj
    
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value) #calling original __setattr__ to override special methods
        else:
            setattr(self._obj, name, value)

In [1]:
#8.8 Extending a Property in a Subclass

class Team:
    def __init__(self, team_name):
        self.team_name = team_name
    
    #Getter function
    @property
    def team_name(self):
        return self._team_name
    
    #Setter function
    @team_name.setter
    def team_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._team_name = value
        
    #Deleter function (optional) 
    @team_name.deleter
    def team_name(self):
        raise AttributeError('Cannot delete attribute')

In [5]:
class SubTeam(Team):
    @property
    def team_name(self):
        print('Getting team name')
        return super().team_name
    
    @team_name.setter
    def team_name(self, value):
        print('Setting team name to', value)
        super(SubTeam, SubTeam).team_name.__set__(self, value)
        
    @team_name.deleter
    def team_name(self):
        print('Deleting team name')
        super(SubTeam, SubTeam).team_name.__delete__(self)

In [6]:
team5 = SubTeam('San Jose Sharks')

Setting team name to San Jose Sharks


In [7]:
team5.team_name

Getting team name


'San Jose Sharks'

###8.9 Creating a New Kind of Class or Instance Attribute

In [8]:
#To create an entirely new kind of instance attribute, we have to define its functionality in the for of a descriptor class
#Descriptor attribute for an integer type-checked attribute

class Integer:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

```
A descriptor is a class that implements the three core attribute access operations (get,
set, and delete) in the form of __get__(), __set__(), and __delete__() special meth‐
ods. These methods work by receiving an instance as input. The underlying dictionary
of the instance is then manipulated as appropriate.
```

In [9]:
#To use a descriptor, instances of the descriptor are placed into a class definition as class variables

class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [10]:
p = Point(5, 6)

In [11]:
p.x    #Calls Point.x.__get__(p,Point)

5

In [12]:
p.y    #Calls Point.y.__get__(p,Point)

6

In [13]:
p.x = 17 #Calls Point.x.__set__(p, 17)
p.x

17

###8.10 Using Lazily Computed Properties

In [14]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__,value)
            return value

In [15]:
#to use the above code, we use it in a class

import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius **2
    
    @lazyproperty
    def perimeter(self):
        print('Computing area')
        return 2 * math.pi * self.radius

In [16]:
c = Circle(6.0)
c.radius

6.0

In [17]:
c.area

Computing area


113.09733552923255

In [18]:
c.perimeter

Computing area


37.69911184307752